# Get Data

In [1]:
import requests
import json
import pandas as pd
import urllib.parse

In [2]:
api_key = "enter your uspto patent view API Key"

In [ ]:
def fetch_page(query_url, page, api_key):
    options = {"page": page, "per_page": 100, "sort": [{"patent_year": "asc"}]}
    url = query_url + '&f=' + json.dumps(fields) + '&o=' + json.dumps(options)
    response = requests.get(url, headers={"X-Api-Key": api_key})
    if response.status_code == 200:
        return response.json()
    else:
        return None

# Your existing code to build the query
query = {
    "_and": [
        {"_gte": {"patent_date": "2000-01-01"}},
        {"_lte": {"patent_date": "2020-12-31"}},
        {"_or": [
            {"_text_phrase": {"patent_title": "water"}},
            {"_text_phrase": {"patent_abstract": "water"}}
        ]},
        {
            "_or": [
                {"_text_any": {"patent_title": "desalination desalinate desalinating sea salin salt brackish"}},
                {"_text_any": {"patent_abstract": "desalination desalinate desalinating sea salin salt brackish"}}
                ]
        },
        {
            "_or":[
                {"_and": [
                    {"_eq": {"ipc_section": "C"}},
                    { "_or":[
                        {"_eq": {"ipc_class": "02"}},
                        {"_eq": {"ipc_class": "2"}},
                    ]},
                    {"_eq": {"ipc_subclass": "F"}}
                    ]
                },
                {"_and": [
                    {"_eq": {"ipc_section": "B"}},
                    {"_or":[
                        {"_eq": {"ipc_class": "01"}},
                        {"_eq": {"ipc_class": "1"}},
                    ]},
                    {"_eq": {"ipc_subclass": "D"}}
                    ]
                },
                {"_and": [
                    {"_eq": {"ipc_section": "B"}},
                    {"_or":[
                        {"_eq": {"ipc_class": "01"}},
                        {"_eq": {"ipc_class": "1"}},
                    ]},
                    {"_eq": {"ipc_subclass": "J"}}
                    ]
                },
                {"_and": [
                    {"_eq": {"ipc_section": "F"}},
                    {"_or":[
                        {"_eq": {"ipc_class": "04"}},
                        {"_eq": {"ipc_class": "4"}},
                    ]},
                    {"_eq": {"ipc_subclass": "B"}}
                    ]
                },
                {"_and": [
                    {"_eq": {"ipc_section": "F"}},
                    {"_eq": {"ipc_class": "24"}},
                    {"_eq": {"ipc_subclass": "J"}}
                    ]
                }
            ]
        }
        ]
}
fields = ["patent_number", "patent_year", "patent_title","patent_abstract",
          "inventor_country","assignee_country","assignee_organization","assignee_type",
          "cited_patent_number","cited_patent_date",
          "citedby_patent_number","citedby_patent_date",
          "ipc_section", "ipc_class","ipc_subclass","ipc_main_group", "ipc_subgroup"]
query_string = json.dumps(query)
encoded_query = urllib.parse.quote(query_string)

base_url = 'https://api.patentsview.org/patents/query?q='
query_url = base_url + encoded_query
# Initialize an empty list to store DataFrames
dataframes = []

# Initialize a variable for the current page
current_page = 1

# Loop to fetch each page
while True:
    data = fetch_page(query_url, current_page, api_key)
    if data and data.get('patents'):
        # Convert the data to a DataFrame
        df = pd.DataFrame(data.get('patents'))
        dataframes.append(df)
        
        # Increment the page number for the next request
        current_page += 1
    else:
        # Exit the loop if there are no more results
        break

# Concatenate all DataFrames into a single DataFrame
all_patents = pd.concat(dataframes, ignore_index=True)

# Save the complete DataFrame to a CSV file
csv_file_path = 'filename.csv'
all_patents.to_csv(csv_file_path, index=False, encoding='utf-8')
print(f"Data successfully saved to {csv_file_path}")